In [2]:
!pip install langchain openai --upgrade --quiet

### 基于OpenAI的Python SDK使用函数调用

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
model = 'gpt-3.5-turbo-0613'

In [5]:
function_descriptions = [
  {
      "name": "get_student_score",
      "description": "Get the student score by given his or her name",
      "parameters": {
          "type": "object",
          "properties": {
              "name": {
                  "type": "string",
                  "description": "The student's name",
              }
          },
          "required": ["name"],
      },
    }
]

In [6]:
user_query = "What's the performance of Lucy in the school this year?"

In [7]:
from openai import OpenAI

client = OpenAI()
response = client.chat.completions.create(model=model,
                                          messages=[{"role": "user", "content": user_query}],
                                          functions=function_descriptions,
                                          function_call="auto")
ai_response_message = response.choices[0].message
print(ai_response_message)
name = eval(ai_response_message.function_call.arguments).get("name")
name

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "name": "Lucy"\n}', name='get_student_score'), tool_calls=None)


'Lucy'

In [8]:
import json

SCORES = { 'Alex': 90, 'Lucy': 60 }
def get_student_score(name):

    """Get the student score by given his or her name"""

    score = {
        "name": name,
        "score": SCORES[name]
    }
    return json.dumps(score)

function_response = get_student_score(name=name)
function_response

'{"name": "Lucy", "score": 60}'

In [9]:
second_response = client.chat.completions.create(model=model,
                                                 messages=[
                                                     {"role": "user", "content": user_query},
                                                     ai_response_message,
                                                     {
                                                         "role": "function",
                                                         "name": "get_student_score",
                                                         "content": function_response,
                                                     },
                                                 ])
print (second_response.choices[0].message.content)

The performance of Lucy in school this year is a score of 60.
